# Imports

In [1]:
import pandas as pd
import numpy as np

In [19]:
# df = pd.DataFrame(
#     [['electricity','DLinear','feature_ablation',1,'0:01:19.521756']],
#     columns=['dataset', 'model', 'attr_method', 'itr_no', 'time']
# )
# df['time'] = pd.to_timedelta(df['time'])
# df['time'].dt.total_seconds()

In [19]:
def reduce_df(df:pd.DataFrame):
    # df[df['area']==0.05]
    return df.groupby('metric').aggregate('mean')[['comp', 'suff']].reset_index()

int_metric_map = {
    'electricity': ['mae'],
    'traffic': ['mae'],
    'mimic_iii': ['auc']
}

test_metric_map = {
    'electricity': ['mae', 'mse'],
    'traffic': ['mae', 'mse'],
    'mimic_iii': ['auc', 'accuracy']
}

datasets = ['electricity', 'traffic', 'mimic_iii']
models = ['DLinear', 'MICN', 'SegRNN', 'iTransformer']
attr_methods = [
    'feature_ablation', 'augmented_occlusion', 
    'feature_permutation', 
    'integrated_gradients', 'gradient_shap','dyna_mask',
    'winIT', 'tsr' ,'wtsr'
]

short_form = {
    'feature_ablation': 'FA',
    'occlusion':'FO',
    'augmented_occlusion': 'AFO',
    'feature_permutation': 'FP',
    'winIT': 'WinIT',
    'tsr':'TSR',
    'wtsr': 'WinTSR',
    'gradient_shap': 'GS',
    'integrated_gradients': 'IG',
    'dyna_mask': 'DM'
}
NUM_ITERATIONS = 3

# Summarize

In [21]:
results = []
for dataset in datasets:
    for attr_method in attr_methods:
        for metric in int_metric_map[dataset]:
            for model in models:
                for itr_no in range(1, NUM_ITERATIONS+1):
                    df = pd.read_csv(f'results/{dataset}_{model}/{itr_no}/{attr_method}.csv')
                    df = reduce_df(df)
                    comp, suff= df[df['metric']==metric][['comp', 'suff']].values[0]
                    
                    results.append([
                        dataset, attr_method, metric, model, itr_no, comp, suff
                    ])

result_df = pd.DataFrame(results, columns=['dataset', 'attr_method', 'metric', 'model', 'itr_no', 'comp', 'suff'])
result_df.head(3)

,dataset,attr_method,metric,model,itr_no,comp,suff
0,electricity,feature_ablation,mae,DLinear,1,10.860362,12.558042
1,electricity,feature_ablation,mae,DLinear,2,10.433986,12.210812
2,electricity,feature_ablation,mae,DLinear,3,10.852448,12.406176


In [22]:
result_df = result_df.groupby(['dataset', 'attr_method', 'metric', 'model'])[['comp', 'suff']].mean().reset_index()
result_df = result_df[result_df['metric'].isin(['mae', 'auc'])]

selected = result_df['metric'].isin(['auc', 'accuracy'])
result_df.loc[selected, ['comp', 'suff']] = 1 - result_df[selected][['comp', 'suff']]

In [23]:
result_df['comp_rank'] = result_df.groupby(['dataset', 'metric', 'model'])['comp'].rank(ascending=False)
result_df['suff_rank'] = result_df.groupby(['dataset', 'metric', 'model'])['suff'].rank(ascending=True)
result_df.groupby(['dataset', 'metric', 'attr_method'])[['comp_rank', 'suff_rank']].mean().reset_index()

,dataset,metric,attr_method,comp_rank,suff_rank
0,electricity,mae,augmented_occlusion,5.25,4.00
1,electricity,mae,dyna_mask,9.00,7.75
2,electricity,mae,feature_ablation,3.50,3.00
3,electricity,mae,feature_permutation,7.75,6.50
4,electricity,mae,gradient_shap,7.00,7.50
5,electricity,mae,integrated_gradients,3.00,8.25
6,electricity,mae,tsr,2.25,2.00
7,electricity,mae,winIT,6.00,5.00
8,electricity,mae,wtsr,1.25,1.00
9,mimic_iii,auc,augmented_occlusion,5.50,2.50


In [24]:
result_df.groupby(['dataset', 'metric', 'attr_method'])[['comp_rank', 'suff_rank']].std().round(1).reset_index()

,dataset,metric,attr_method,comp_rank,suff_rank
0,electricity,mae,augmented_occlusion,0.5,0.0
1,electricity,mae,dyna_mask,0.0,1.0
2,electricity,mae,feature_ablation,0.6,0.0
3,electricity,mae,feature_permutation,0.5,1.0
4,electricity,mae,gradient_shap,0.8,0.6
5,electricity,mae,integrated_gradients,1.4,1.5
6,electricity,mae,tsr,0.5,0.0
7,electricity,mae,winIT,0.8,0.0
8,electricity,mae,wtsr,0.5,0.0
9,mimic_iii,auc,augmented_occlusion,1.3,1.9


In [25]:
df = pd.concat([
    result_df.drop(columns='suff_rank').rename(columns={'comp_rank': 'rank'}), 
    result_df.drop(columns='comp_rank').rename(columns={'suff_rank': 'rank'})
], axis=0)

In [30]:
ranks = df.groupby(['dataset', 'metric', 'attr_method'])['rank'].mean().round(1).reset_index(name='mean_rank')
ranks['rank'] = ranks.groupby(['dataset', 'metric'])['mean_rank'].rank()
ranks

,dataset,metric,attr_method,mean_rank
0,electricity,mae,augmented_occlusion,4.6
1,electricity,mae,dyna_mask,8.4
2,electricity,mae,feature_ablation,3.2
3,electricity,mae,feature_permutation,7.1
4,electricity,mae,gradient_shap,7.2
5,electricity,mae,integrated_gradients,5.6
6,electricity,mae,tsr,2.1
7,electricity,mae,winIT,5.5
8,electricity,mae,wtsr,1.1
9,mimic_iii,auc,augmented_occlusion,4.0


,dataset,metric,attr_method,mean_rank,rank
0,electricity,mae,augmented_occlusion,4.6,4.0
1,electricity,mae,dyna_mask,8.4,9.0
2,electricity,mae,feature_ablation,3.2,3.0
3,electricity,mae,feature_permutation,7.1,7.0
4,electricity,mae,gradient_shap,7.2,8.0
5,electricity,mae,integrated_gradients,5.6,6.0
6,electricity,mae,tsr,2.1,2.0
7,electricity,mae,winIT,5.5,5.0
8,electricity,mae,wtsr,1.1,1.0
9,mimic_iii,auc,augmented_occlusion,4.0,4.0


In [29]:
df.groupby(['dataset', 'metric', 'attr_method'])['rank'].std().round(2).reset_index()

,dataset,metric,attr_method,mean_rank
0,electricity,mae,augmented_occlusion,0.74
1,electricity,mae,dyna_mask,0.92
2,electricity,mae,feature_ablation,0.46
3,electricity,mae,feature_permutation,0.99
4,electricity,mae,gradient_shap,0.71
5,electricity,mae,integrated_gradients,3.11
6,electricity,mae,tsr,0.35
7,electricity,mae,winIT,0.76
8,electricity,mae,wtsr,0.35
9,mimic_iii,auc,augmented_occlusion,2.20


# Time efficiency

In [12]:
df = pd.read_csv('results/time_efficiency.csv')
df.head(3)

,dataset,model,iteration,name,timestamp,duration
0,traffic,DLinear,1,feature_ablation,2024-07-26 00:28:04.747253,0:01:13.289547
1,electricity,DLinear,1,feature_ablation,2024-07-26 00:28:09.959615,0:01:49.416156
2,mimic_iii,DLinear,1,feature_ablation,2024-07-26 00:29:14.800882,0:01:51.229460


In [13]:
df['seconds'] = pd.to_timedelta(df['duration']).dt.seconds

In [16]:
df = df.groupby(['dataset', 'model', 'name'])['seconds'].mean().reset_index()

In [17]:
df

,dataset,model,name,seconds
0,electricity,DLinear,augmented_occlusion,118.333333
1,electricity,DLinear,dyna_mask,86.333333
2,electricity,DLinear,feature_ablation,109.666667
3,electricity,DLinear,feature_permutation,114.666667
4,electricity,DLinear,gradient_shap,89.000000
...,...,...,...,...
115,traffic,iTransformer,integrated_gradients,189.333333
116,traffic,iTransformer,occlusion,216.666667
117,traffic,iTransformer,tsr,3202.000000
118,traffic,iTransformer,winIT,249.000000


# Variable Lookback

In [20]:
datasets

['electricity', 'traffic', 'mimic_iii']

In [24]:
lookbacks = {
    'electricity':[24, 48], 
    'traffic': [24, 48], 
    'mimic_iii': [24, 36]
}
model = 'iTransformer'
results = []
for dataset in datasets:
    for attr_method in attr_methods[-3:]:
        for metric in int_metric_map[dataset]:
            for lookback in lookbacks[dataset]:
                for itr_no in range(1, NUM_ITERATIONS+1):
                    df = pd.read_csv(f'results/{dataset}_{model}_sl_{lookback}/{itr_no}/{attr_method}.csv')
                    df = reduce_df(df)
                    comp, suff= df[df['metric']==metric][['comp', 'suff']].values[0]
                    
                    results.append([
                        dataset, attr_method, metric, lookback, itr_no, comp, suff
                    ])

result_df = pd.DataFrame(results, columns=['dataset', 'attr_method', 'metric', 'lookback', 'itr_no', 'comp', 'suff'])
result_df.head(3)

,dataset,attr_method,metric,lookback,itr_no,comp,suff
0,electricity,winIT,mae,24,1,14.431164,24.175662
1,electricity,winIT,mae,24,2,14.574075,23.334677
2,electricity,winIT,mae,24,3,14.136588,21.409937


In [27]:
result_df = result_df.groupby(['dataset', 'attr_method', 'metric', 'lookback'])[['comp', 'suff']].mean().reset_index()
result_df

,dataset,attr_method,metric,lookback,comp,suff
0,electricity,tsr,mae,24,17.238143,21.062192
1,electricity,tsr,mae,48,14.255044,15.608042
2,electricity,winIT,mae,24,14.380609,22.973425
3,electricity,winIT,mae,48,11.472749,17.859977
4,electricity,wtsr,mae,24,17.876141,20.678208
5,electricity,wtsr,mae,48,14.851056,15.264080
6,mimic_iii,tsr,auc,24,0.418306,0.389257
7,mimic_iii,tsr,auc,36,0.500166,0.521904
8,mimic_iii,winIT,auc,24,0.450934,0.370624
9,mimic_iii,winIT,auc,36,0.548548,0.447009


In [28]:
result_df[result_df['dataset']=='electricity']

,dataset,attr_method,metric,lookback,comp,suff
0,electricity,tsr,mae,24,17.238143,21.062192
1,electricity,tsr,mae,48,14.255044,15.608042
2,electricity,winIT,mae,24,14.380609,22.973425
3,electricity,winIT,mae,48,11.472749,17.859977
4,electricity,wtsr,mae,24,17.876141,20.678208
5,electricity,wtsr,mae,48,14.851056,15.264080
